In [1]:
%cd /home/druhe/rail1/src/cnf/
import os
os.environ['DATAROOT'] = '/home/druhe/datasets/'

/home/druhe/rail1/src/cnf


/home/druhe/rail1/src/cnf/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import torch
from torch import nn, optim
import torch.nn.functional as F
import torch.nn.init

In [3]:
from datasets import embedded_mnist


data = embedded_mnist.load_embedded_mnist('runs/notebooks/9utazgqo/latent_space/', subset=None, batch_size=128)

In [130]:
dataloader = data['train_loader']

In [131]:
class FiLMLinear(nn.Module):
    def __init__(self, in_features, out_features, c_features):
        super(FiLMLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.c_features = c_features

        self.linear = nn.Linear(in_features, out_features)
        self.gamma = nn.Linear(c_features, out_features)
        self.beta = nn.Linear(c_features, out_features)

        # Fill gamma and beta with zeros.
        torch.nn.init.zeros_(self.gamma.weight)
        torch.nn.init.zeros_(self.gamma.bias)

        torch.nn.init.zeros_(self.beta.weight)
        torch.nn.init.zeros_(self.beta.bias)

    def forward(self, x, c):

        x = self.linear(x)
        return x * (1 + self.gamma(c)) + self.beta(c)

In [132]:
class NeuralField(nn.Module):
    def __init__(
        self,
        input_size,
        hidden_size,
        output_size,
        c_size,
        num_layers=8,
        activation=nn.GELU(),
    ):
        super(NeuralField, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.c_size = c_size
        self.num_layers = num_layers
        self.activation = activation

        self.c_emb = nn.Linear(c_size, hidden_size)

        self.layers = nn.ModuleList()
        for i in range(num_layers):
            self.layers.append(FiLMLinear(input_size, hidden_size, hidden_size))
            input_size = hidden_size

        self.output = nn.Linear(hidden_size, output_size)

    def forward(self, x, c):

        c = self.activation(self.c_emb(c))

        for layer in self.layers:
            x = self.activation(layer(x, c))
        return self.output(x)



In [133]:
example = next(iter(dataloader))


In [134]:
def sample_random_locations(image_tensor, n=1024):
    batch_size = len(image_tensor)
    coordinates = torch.rand(batch_size, n, 2, device='cuda') * 2 - 1
    targets = F.grid_sample(image_tensor, coordinates.unsqueeze(2), align_corners=True, mode='nearest')
    return coordinates, targets





In [135]:
num_locs_per_sample = 512
model = NeuralField(2, 128, 1, 128)
model = model.cuda()
adam = optim.Adam(model.parameters(), lr=1e-4)
while True:
    for i, (x, _, z) in enumerate(dataloader):
        x = x.cuda()
        z = z.cuda()

        adam.zero_grad()

        coordinates, targets = sample_random_locations(x, num_locs_per_sample)
        z = z[:, None].expand(-1, num_locs_per_sample, -1)
        y_hat = model(coordinates, z)


        
        loss = nn.functional.mse_loss(y_hat, targets)
        loss.backward()
        adam.step()
        if i % 16 == 0:
            print(loss.item())

0.12887021899223328
0.09909767657518387
0.09826815873384476
0.10212045162916183
0.09633906930685043
0.09657901525497437
0.0968041941523552
0.08392412215471268
0.07348500192165375
0.07064205408096313
0.06822293251752853
0.06628334522247314
0.06448803842067719
0.06385192275047302
0.062129586935043335
0.06101730465888977
0.06041358411312103
0.0580119714140892
0.056791406124830246
0.051271952688694
0.05379011482000351
0.052354756742715836
0.050430234521627426
0.047877486795186996
0.04987746849656105
0.046171486377716064
0.04530791938304901
0.042948685586452484
0.03953826427459717
0.04143570736050606
0.04059959948062897
0.041205938905477524
0.0418359637260437
0.0395723357796669
0.03859113156795502
0.03815428540110588
0.03672134876251221
0.0364784374833107
0.03822949901223183
0.0366065576672554
0.03575369715690613
0.035469405353069305
0.031768567860126495
0.0327334925532341
0.03305833786725998
0.03333194553852081
0.03391990065574646
0.0317830964922905
0.03118932992219925
0.03215823695063591


KeyboardInterrupt: 